In [1]:
import hindbrain as hb
import numpy as np
import struct
from array import array

In [2]:
# mnist dataset preparation
train_labels_path = 'examples/mnist/train-labels.idx1-ubyte'
train_images_path = 'examples/mnist/train-images.idx3-ubyte'

test_labels_path = 'examples/mnist/t10k-labels.idx1-ubyte'
test_images_path = 'examples/mnist/t10k-images.idx3-ubyte'

In [3]:
# function for loading labels data from file
def load_data(filepath):
    with open(file=filepath, mode='rb') as file:
        magic, size = struct.unpack(">II", file.read(8))
        if magic != 2049:
            raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
        labels = array("B", file.read())  
    return np.asarray(labels)

# function for loading images from file
def load_image_data(filepath):
    with open(file=filepath, mode='rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        if magic != 2051:
            raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
        image_data = array("B", file.read())        
    images = []
    for i in range(size):
        images.append([0] * rows * cols)
    for i in range(size):
        img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
        img = img.reshape(28, 28)
        images[i][:] = img      
    return np.asarray(images)

In [4]:
# training dataset
train_labels = load_data(train_labels_path)
train_images = load_image_data(train_images_path)

train_images_scaled = train_images/255

# testing dataset

test_labels = load_data(test_labels_path)
test_images = load_image_data(test_images_path)

test_images_scaled = test_images/255

train_labels_one_hot = hb.one_hot(train_labels, depth=10)

In [5]:
# neural network model with hindbrain pakage

mnist_model = hb.Model()

mnist_model.add_layer(hb.InputLayer(784, flatten=True))
mnist_model.add_layer(hb.LinearLayer(256), activation='tanh')
mnist_model.add_layer(hb.LinearLayer(256), activation='tanh')
mnist_model.add_layer(hb.LinearLayer(256), activation='tanh')
mnist_model.add_layer(hb.LinearLayer(10), activation='softmax')

In [6]:
mnist_model.summary()

Total number of layers:  5
Model name:  Default_model
Layer number: 0, layer name: input_layer, number of neurons: 784, activation: None
Layer number: 1, layer name: linear_layer, number of neurons: 256, activation: tanh
Layer number: 2, layer name: linear_layer, number of neurons: 256, activation: tanh
Layer number: 3, layer name: linear_layer, number of neurons: 256, activation: tanh
Layer number: 4, layer name: linear_layer, number of neurons: 10, activation: softmax
Loss: None


In [7]:
mnist_model.build(loss='categorical_cross_entropy', optimizer='SGD', learning_rate=0.002, momentum=0.9, beta=0.999)

In [8]:
for i in range(6):
    for data, label in zip(train_images_scaled, train_labels_one_hot):
        mnist_model.train(data, label, epochs=1)

loss 3.716069
loss 2.113892
loss 3.113465
loss 1.303038
loss 5.182793
loss 1.946467
loss 0.720245
loss 4.457473
loss 0.663117
loss 5.500924
loss 3.859479
loss 3.637326
loss 2.650795
loss 0.786864
loss 0.498124
loss 2.248043
loss 2.200405
loss 3.434273
loss 4.702209
loss 3.287971
loss 3.815333
loss 3.227952
loss 2.567795
loss 1.51386
loss 4.237208
loss 1.837967
loss 1.045944
loss 3.138532
loss 2.894735
loss 3.605834
loss 2.247353
loss 0.923639
loss 2.127584
loss 4.717129
loss 2.54891
loss 5.83142
loss 0.981109
loss 2.962205
loss 5.374894
loss 0.900298
loss 0.577374
loss 2.854437
loss 2.340318
loss 2.811025
loss 1.974721
loss 3.882165
loss 3.39713
loss 3.274842
loss 3.320829
loss 3.739154
loss 2.1587
loss 2.976505
loss 3.749653
loss 2.493082
loss 3.121477
loss 2.681952
loss 2.715338
loss 1.83485
loss 3.347662
loss 1.421455
loss 4.858682
loss 1.390968
loss 1.395627
loss 1.138311
loss 4.209377
loss 2.846458
loss 1.316394
loss 0.568637
loss 1.806226
loss 0.915532
loss 1.71291
loss 2.298294


KeyboardInterrupt: 

In [ ]:
c = 0
for z in range(10000):
    a = np.argmax(mnist_model.predict(test_images_scaled[z]))
    b = test_labels[z]
    if a == b:
        c += 1
        
print(f'Accuracy: {c/10000}')